In [1]:
import box

In [2]:
import os

In [3]:
%pwd

'/Users/abcom/Documents/workingArea/pythonProjects/KidneyDisease/KidneyDiseaseClassification/research'

In [4]:
os.chdir('../') 

In [5]:
%pwd

'/Users/abcom/Documents/workingArea/pythonProjects/KidneyDisease/KidneyDiseaseClassification'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier import logger

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            logger.info("yaml config file: {config_filepath} loaded successfully")
            self.params = read_yaml(params_filepath)
            logger.info("yaml params file: {params_filepath} loaded successfully")

            create_directories([self.config.artifacts_root])
            logger.info("create_directories created successfully")

    def get_data_ingestion_config(self) -> DataIngestionConfig:
            config = self.config.data_ingestion

            create_directories([config.root_dir])

            data_ingestion_config = DataIngestionConfig(
                  root_dir= config.root_dir,
                  source_URL=config.source_URL,
                  local_data_file=config.local_data_file,
                  unzip_dir= config.unzip_dir
            )
            return data_ingestion_config

In [9]:
import os
import zipfile
import gdown
from cnnClassifier  import logger
from cnnClassifier.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self)-> str:
        
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix ="https://drive.google.com/uc?export=download&id="
            gdown.download(prefix+file_id, zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-05 13:38:39,756: INFO: common: yaml file: {path_to_yaml} loaded successfully]
[2024-10-05 13:38:39,757: INFO: 3803003287: yaml config file: {config_filepath} loaded successfully]
[2024-10-05 13:38:39,759: INFO: common: yaml file: {path_to_yaml} loaded successfully]
[2024-10-05 13:38:39,760: INFO: 3803003287: yaml params file: {params_filepath} loaded successfully]
[2024-10-05 13:38:39,760: INFO: common: created directory at: artifacts]
[2024-10-05 13:38:39,761: INFO: 3803003287: create_directories created successfully]
[2024-10-05 13:38:39,762: INFO: common: created directory at: artifacts/data_ingestion]
[2024-10-05 13:38:39,763: INFO: 3764345020: Downloading data from https://drive.google.com/file/d/1FvQL9V5QXtNJtFreSb21m2B1DQWECMt2/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?export=download&id=1FvQL9V5QXtNJtFreSb21m2B1DQWECMt2
From (redirected): https://drive.google.com/uc?export=download&id=1FvQL9V5QXtNJtFreSb21m2B1DQWECMt2&confirm=t&uuid=5e94adba-21b3-4c2d-b990-1b719c66b483
To: /Users/abcom/Documents/workingArea/pythonProjects/KidneyDisease/KidneyDiseaseClassification/artifacts/data_ingestion/data.zip
100%|██████████| 943M/943M [01:29<00:00, 10.6MB/s] 

[2024-10-05 13:40:14,710: INFO: 3764345020: Downloaded data from https://drive.google.com/file/d/1FvQL9V5QXtNJtFreSb21m2B1DQWECMt2/view?usp=sharing into file artifacts/data_ingestion/data.zip]
